<a href="https://colab.research.google.com/github/Abhirrami/sdc/blob/main/CAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q gradio tensorflow nltk

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import gradio as gr
import nltk

nltk.download("punkt")

# Sample training data
texts = [
    "AI is transforming the technology industry",
    "The economy is facing inflation pressures",
    "The team won the championship this year",
    "Google introduces new hardware products",
    "Government introduces new tax reform",
    "The player scored a record number of goals"
]
labels = ["tech", "business", "sports", "tech", "politics", "sports"]
label_map = {label: idx for idx, label in enumerate(set(labels))}
reverse_label_map = {idx: label for label, idx in label_map.items()}
y = np.array([label_map[label] for label in labels])

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
maxlen = max(len(seq) for seq in sequences)
vocab_size = len(tokenizer.word_index) + 1
X = pad_sequences(sequences, maxlen=maxlen)

# Simple attention layer
class Attention(Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="random_normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros", trainable=True)
        super().build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

# Build model
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 64)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention_layer = Attention()(lstm_layer)
output_layer = Dense(len(label_map), activation='softmax')(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
model.fit(X, y, epochs=10, verbose=1)

# Inference function
def classify_news(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=maxlen)
    prediction = model.predict(padded)[0]
    predicted_label = reverse_label_map[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    return f"Predicted Category: {predicted_label.capitalize()} ({confidence:.2f}%)"

# Gradio UI
gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(lines=3, placeholder="Enter a news headline here..."),
    outputs="text",
    title="🧠 (CAN) News Category Classifier with Attention",
    description="Enter a news headline, and the model will classify it into a category like tech, politics, business, or sports using LSTM with attention."
).launch()
